In [1]:
# conda activate nilearn37

In [2]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import nibabel as nib
from nilearn import plotting, datasets, surface, image
import seaborn as sns

from src.regress import ridge

from sklearn.decomposition import PCA
from scipy.stats import pearsonr

In [3]:
# from mayavi import mlab
# from surfer import Brain

In [4]:
def mkNifti(arr, mask, im, nii=True):
    out_im = np.zeros(mask.size, dtype=arr.dtype)
    inds = np.where(mask)[0]
    out_im[inds] = arr
    if nii:
        out_im = out_im.reshape(im.shape)
        out_im = nib.Nifti1Image(out_im, affine=im.affine)
    return out_im

In [5]:
process = 'components'
top_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis'
data_dir = f'{top_dir}/data/raw'
out_dir = f'{top_dir}/data/interim'
figure_dir = f'{top_dir}/reports/figures/{process}'
if not os.path.exists(figure_dir):
    os.mkdir(figure_dir)

In [6]:
#Load mask
mask = np.load(f'{out_dir}/Reliability/sub-all_reliability-mask.npy').astype('bool')
im = nib.load(f'{out_dir}/Reliability/sub-all_stat-rho_statmap.nii.gz')

rs = np.load(f'{out_dir}/VoxelPermutation/sub-all/sub-all_feature-all_rs-filtered.npy').astype('bool')
rs = mkNifti(rs, mask, im, nii=False)

In [8]:
base = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim/VoxelPermutation/sub-all/sub-all_feature-XXX_rs.npy'
pred = []
for feature in features:
    arr = np.load(base.replace('XXX', feature))
    arr = np.expand_dims(arr, axis=1)
    if type(pred) is list:
        pred = arr
    else:
        pred = np.hstack([pred, arr])
preference = np.argmax(pred, axis=1)

In [9]:
for i in range(n_features):
    print(f'Percent voxels for feature: {(np.sum(preference == i)/len(preference))*100:0.1f}')

Percent voxels for feature: 6.7
Percent voxels for feature: 14.7
Percent voxels for feature: 14.3
Percent voxels for feature: 4.9
Percent voxels for feature: 12.3
Percent voxels for feature: 2.8
Percent voxels for feature: 12.3
Percent voxels for feature: 3.1
Percent voxels for feature: 4.3
Percent voxels for feature: 7.9
Percent voxels for feature: 3.2
Percent voxels for feature: 13.3


In [ ]:
vmin = 1
vmax = len(np.unique(preference)) - 1 + vmin
preference += vmin

In [ ]:
preference = mkNifti(preference, mask, im, nii=False)
preference[~rs] = 0
preference = preference.astype('float')
preference = nib.Nifti1Image(preference.reshape(im.shape), affine=im.affine)

In [60]:
view = plotting.view_img(preference, cmap=cmap,
                        threshold=vmin, vmax=vmax,
                        symmetric_cmap=False)
view.open_in_browser()

In [61]:
fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage')
texture = {'left': [], 'right': []}
for hemi in ['left', 'right']: 
    arr = surface.vol_to_surf(preference, fsaverage[f'pial_{hemi}'],
                                       interpolation='nearest')
    texture[hemi] = arr.astype('int')
    
for hemi in ['left', 'right']:
    view = plotting.view_surf(fsaverage[f'infl_{hemi}'],
                              title=hemi,
                              surf_map=texture[hemi],
                              symmetric_cmap=False,
                              vmax=vmax,
                              threshold=vmin,
                              cmap=cmap,bg_map=fsaverage[f'sulc_{hemi}'])
    view.open_in_browser()